In [50]:
#Reference : https://www.tensorflow.org/text/tutorials/text_generation


import tensorflow as tf
from tensorflow import keras
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib.pyplot as plt

In [51]:
filepath =r"C:\Users\hp\Desktop\ML Models\RNNs\shakespeare.txt"
with open(filepath) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])



First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) #char_level=True means that we are tokenizing characters instead of words
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
print(tokenizer.texts_to_sequences(["First"]))
print(tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]]))


[[20, 6, 9, 8, 3]]
['f i r s t']


In [5]:
max_id = len(tokenizer.word_index) #total number of distinct characters
dataset_size = tokenizer.document_count #total number of characters
print(max_id, dataset_size)

39 1115394


In [6]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # we substract 1 to get IDs from 0 to 38 rather than from 1 to 39 becuase we will use the IDs as targets, and we want them to start from 0
print(encoded)

[19  5  8 ... 20 26 10]


In [7]:
#taking 90% of dataset as training data
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
#above "from_tensor_slices"  creates a seperate tensor dataset from each element of the tensor encoded


In [8]:
dataset = dataset.repeat().window(101,shift=1,drop_remainder=True)   
#repeat() method basically continues the data until the window is filled
#exammple: if data is [1,2,3] and window is 4 then repeat() will make it [1,2,3,1] [2,3,1,2]

In [9]:
dataset = dataset.flat_map(lambda window: window.batch(101))
 #It’s a nested dataset, analogous to a list of lists. However, we cannot use a nested dataset directly for training, as our model will expect tensors as input, not datasets. So, we must call the flat_map() method: it converts a nested dataset into a flat dataset.


In [10]:
np.random.seed(42)
tf.random.set_seed(42)

In [11]:

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [12]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [13]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [14]:
#Dropout is a regularization technique where a certain proportion of randomly selected neurons are "dropped out" 
#or ignored during training. This means that their contribution to the next layer is temporarily removed.
# in this case the drouput rate is 0.2 (its normaly between 0-1) 

#Recurrent Dropout is a variant of dropout specifically designed for recurrent neural networks. 
#In RNNs like GRUs or LSTMs, dropout is applied not only to the input units of a layer but also to 
#the recurrent connections within the layer.


model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [ ]:
#This will take a while to run
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                   epochs=10)
model.save("shakespeare.h5")

In [31]:
model = keras.models.load_model(r"C:\Users\hp\Desktop\ML Models\RNNs\shakespeare.h5")

# creating functions to preprocvess like earlier for inputs
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

X_new = preprocess(["How are yo"])
print(model.predict(X_new)[0, -1:, :]) 
Y_pred = model.predict(X_new)
print(tokenizer.sequences_to_texts(Y_pred.argmax(axis=-1) + 1)[0][-1])


1/1 [==============================] - 0s 311ms/step
[[3.51145832e-06 1.67668315e-06 3.36331104e-05 1.06268180e-05
  3.22712071e-06 1.95957838e-07 2.18989044e-07 1.37377788e-06
  3.51481722e-05 1.63527249e-04 1.04154836e-07 1.24610847e-06
  9.37553182e-08 9.99730766e-01 1.96858991e-07 6.05944379e-08
  1.56148633e-06 8.94588794e-08 8.60609362e-09 2.82524759e-09
  2.90034950e-06 4.94311223e-08 1.00237652e-09 3.15139630e-08
  4.75855501e-07 1.21086180e-06 1.59099443e-07 1.66274447e-07
  2.29161188e-08 1.34984489e-07 1.76165056e-08 1.62302086e-07
  2.41419906e-09 4.21676472e-10 7.44343379e-06 3.57352425e-10
  1.12415849e-11 1.74161710e-13 1.26173215e-13]]
1/1 [==============================] - 0s 20ms/step
u


In [32]:
def next_char(text, temperature=1):
    X_new = preprocess([text])   #text preprocesss , convert to one hot encoding
    y_proba = model.predict(X_new)[0, -1:, :]    #predicting the next character classes
    rescaled_logits = tf.math.log(y_proba) / temperature  #rescaling the logits
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [33]:
next_char("How are yo", temperature=1)

1/1 [==============================] - 0s 19ms/step


'u'

In [47]:
def complete_text(text, n_chars=250, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
temperatures = [0.5, 1.0, 1.2,1.4]
para_list = []
for temp in temperatures:
    print(f"temperature: {temp}")
    para_list.append((complete_text("Thy", temperature=temp),temp))


In [53]:
para_list = para_list.copy()
for para,temp in para_list:
    print(temp)
    print(para)
    print("\n")

0.5
Thy will accomped
the proper in the charge of such a sight of the belly.

menenius:
you must not give me quarrel.

first citizen:
what can you see the old and common and be to
the corminions of the finger and be this common and suffering of the worst.




1.0
Thy good to the gater,
where he for you can speak the motitutes was he,
are these forthy counternon's look in, where company me the store none whereof they,
for the one still sign trot with answer,
and i say you you studding ovind, well shall be
well-bo


1.2
Thy. this citizens vince
hear myself and wares 'that speil was to!

staten:
sir, humbly, they? when some is give
which yet to provide her belly; have gells.
i pray you, as i reserve as friends,
alone liking, what is the sleve; rock assist you
live in hi


1.4
Thy wiss!

menenius:
if you have appear fit: go die i hus
aursel. nay, what, you shall give toe. forbeon, goe will
if i will barve an over at;
hortensio, thou thou'rt boy, asl, i'll tell
uput a thingf in the rah, the

In [ ]:
# Temperature set to 1 or 1.2 would be the best

In [49]:
test_dataset = tf.data.Dataset.from_tensor_slices(encoded[train_size:])
test_dataset = test_dataset.window(101, shift=1, drop_remainder=True)
test_dataset = test_dataset.flat_map(lambda window: window.batch(101))
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
test_dataset = test_dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
model.evaluate(test_dataset)

3483/3483 [==============================] - 1271s 365ms/step - loss: 1.6047


1.604728102684021